In [1]:
import pandas as pd
import numpy as np
import os
#os.chdir("../")

In [2]:
no_precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="reszta")
precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="Prekambr")

### NIE-PREKAMBR - SYSTEM ERY - JEDNO - ŁATWE

In [54]:
temp=no_precambr_data[["ERA", "SYSTEM"]].drop_duplicates()
n = len(temp)

In [60]:
target_list = temp["ERA"].unique()
target_list = [ele for ele in target_list for i in range(n*2)]
scope_list = temp["SYSTEM"].unique()

In [77]:
question_text = []
possible_answers = []
correct_answers = []

for target in target_list:
    question_text.append(f"Wypisz system ery {target}")
    correct_array = temp[temp["ERA"]==target]["SYSTEM"].unique()
    wrong_array = [element for element in scope_list if element not in correct_array]

    # print(f"CORRECT {correct_array}")
    # print(f"WRONG {wrong_array}")
    # if len(wrong_array)<4:
    #     print('WARNING!!!!!-------------------')
    #     possible_answers.append(np.repeat(np.nan, 5))
    #     correct_answers.append(np.nan)
    #     continue

    temp_possible_answers = np.concatenate(
        (np.random.choice(wrong_array, 3, replace=False), np.random.choice(correct_array, 1))
    )
    np.random.shuffle(temp_possible_answers)
    temp_possible_answers = np.append(temp_possible_answers, np.nan)
    for correct in temp_possible_answers:
        if correct in correct_array:
            correct_answers.append(correct)
            continue

    possible_answers.append(temp_possible_answers)

df1=pd.DataFrame({
    'question':question_text,
    'correct_answer':correct_answers
})

df2=pd.DataFrame(
    possible_answers,
    columns=["A", "B", "C", "D", "E"]
)

df_final = pd.concat([df1, df2], axis=1).drop_duplicates()

In [78]:
df_final

,question,correct_answer,A,B,C,D,E
0,Wypisz system ery kenozoik,czwartorzęd,czwartorzęd,kambr,karbon,trias,NaN
1,Wypisz system ery kenozoik,paleogen,ordowik,perm,paleogen,kambr,NaN
2,Wypisz system ery kenozoik,paleogen,perm,kreda,kambr,paleogen,NaN
3,Wypisz system ery kenozoik,paleogen,trias,jura,karbon,paleogen,NaN
4,Wypisz system ery kenozoik,paleogen,sylur,kreda,paleogen,jura,NaN
...,...,...,...,...,...,...,...
67,Wypisz system ery paleozoik,dewon,dewon,kreda,trias,czwartorzęd,NaN
68,Wypisz system ery paleozoik,dewon,kreda,neogen,paleogen,dewon,NaN
69,Wypisz system ery paleozoik,karbon,karbon,jura,paleogen,neogen,NaN
70,Wypisz system ery paleozoik,kambr,paleogen,trias,kambr,kreda,NaN


In [67]:
possible_answers

[array(['sylur', 'kreda', 'ordowik', 'czwartorzęd', nan], dtype=object),
 array(['perm', 'karbon', 'czwartorzęd', 'sylur', nan], dtype=object),
 array(['czwartorzęd', 'perm', 'trias', 'jura', nan], dtype=object),
 array(['trias', 'karbon', 'czwartorzęd', 'sylur', nan], dtype=object),
 array(['czwartorzęd', 'kambr', 'karbon', 'perm', nan], dtype=object),
 array(['trias', 'paleogen', 'ordowik', 'kambr', nan], dtype=object),
 array(['neogen', 'dewon', 'kreda', 'dewon', nan], dtype=object),
 array(['perm', 'sylur', 'sylur', 'neogen', nan], dtype=object),
 array(['neogen', 'karbon', 'kambr', 'dewon', nan], dtype=object),
 array(['ordowik', 'karbon', 'kambr', 'neogen', nan], dtype=object),
 array(['kreda', 'karbon', 'paleogen', 'jura', nan], dtype=object),
 array(['dewon', 'sylur', 'karbon', 'neogen', nan], dtype=object),
 array(['kreda', 'sylur', 'neogen', 'kambr', nan], dtype=object),
 array(['trias', 'ordowik', 'jura', 'neogen', nan], dtype=object),
 array(['czwartorzęd', 'kreda', 'perm',

In [12]:
data = no_precambr_data
dataPrekambr = precambr_data
isPrekambr = True
isHard = False
startColIdx = 1
endColIdx = 3
multiplyRowsIdx = 2
colName1 = "Eon"
colName2 = "System"
questionPattern = "Prekambr - wybierz system ery"

if isPrekambr:
    df = dataPrekambr.iloc[:, [startColIdx,endColIdx]]
else:
    df = data.iloc[:, [startColIdx,endColIdx]]

df = df.loc[df.index.repeat(multiplyRowsIdx)]

random_answers = []
questions = []

if isHard:
    scope = np.unique(
            np.concatenate((data["PIETRO"].unique(), data["ODDZIAL"].unique(), data["SYSTEM"].unique(), data["ERA"].unique(), 
                            dataPrekambr["Jednostka nieformalna"].unique(), dataPrekambr["Eon"].unique(), 
                            dataPrekambr["Era"].unique(), dataPrekambr["System"].unique()))
                            )

else:
    if isPrekambr:
        scope = dataPrekambr[colName2].unique()
    else:
        scope = data[colName2].unique()

print(scope)

for i in df[colName1]:
    
    if isPrekambr:
        temp_correct = dataPrekambr[dataPrekambr[colName1]==i][colName2].array
    else:
        temp_correct = data[data[colName1]==i][colName2].array

    indexes = np.nonzero(np.in1d(scope, temp_correct))[0]

    print(temp_correct)

    temp_scope = np.delete(scope, indexes)
    
    print(temp_scope)

    if isHard:
        temp = np.concatenate((np.random.choice(temp_scope,4), np.random.choice(temp_correct,1)))
    else:
        temp = np.concatenate((np.random.choice(temp_scope,3), np.random.choice(temp_correct,1)))

    np.random.shuffle(temp)

    if not isHard:
        temp = np.append(temp, np.nan)

    random_answers.append(temp)
    questions.append(f"{questionPattern} {i}")

p1 = pd.DataFrame(questions, columns=["question"])
p2 = pd.DataFrame(random_answers, columns=["A", "B", "C", "D", "E"]).drop_duplicates()
p3 = pd.concat([p1, p2], axis=1)

all_correct_answers = []

for index, row in p3.iterrows():
    temp_val1 = row["question"].split(" ")[-1]

    if temp_val1.isdigit() or temp_val1 in ("górna", "dolna", "środkowa", "górny", "środkowy", "dolny"):
        temp_val1 = row["question"].split(" ")[-2:]
        temp_val1 = " ".join(temp_val1)

    temp_abcde = row[["A", "B", "C", "D", "E"]].tolist()

    if isPrekambr:
        correct_answers = dataPrekambr[dataPrekambr[colName1]==temp_val1][colName2].tolist()
    else:
        correct_answers = data[data[colName1]==temp_val1][colName2].tolist()

    for answer in temp_abcde:
        if answer in correct_answers:
            all_correct_answers.append(answer)
            break

p4 = pd.concat([p3, pd.DataFrame(all_correct_answers, columns=["correct_answer"])], axis=1)

['ediakar' 'kriogen' 'ton' 'sten' 'ektas' 'kalim' 'stater' 'orosir' 'riak'
 'sider' 'brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'ste

In [6]:
p4

,question,A,B,C,D,E,correct_answer
0,Prekambr - wybierz system ery proterozoik,brak,brak,brak,orosir,NaN,orosir
1,Prekambr - wybierz system ery proterozoik,brak,brak,ton,brak,NaN,ton
2,Prekambr - wybierz system ery proterozoik,brak,brak,riak,brak,NaN,riak
3,Prekambr - wybierz system ery proterozoik,brak,brak,orosir,brak,NaN,orosir
4,Prekambr - wybierz system ery proterozoik,brak,brak,brak,sten,NaN,sten
5,Prekambr - wybierz system ery proterozoik,brak,brak,brak,ektas,NaN,ektas
6,Prekambr - wybierz system ery proterozoik,brak,brak,ektas,brak,NaN,ektas
7,Prekambr - wybierz system ery proterozoik,NaN,NaN,NaN,NaN,NaN,stater
8,Prekambr - wybierz system ery proterozoik,stater,brak,brak,brak,NaN,ektas
9,Prekambr - wybierz system ery proterozoik,ektas,brak,brak,brak,NaN,sider
